Katherine Kairis, kak275@pitt.edu, 10/12/2017

In [1]:
from bs4 import BeautifulSoup
import glob
import re

In [2]:
transcripts = glob.glob('data/VOICE/VOICE2.0XML/XML/*.xml')
del transcripts[0]

In [3]:
#Create two dictionaries: one containing information about the participants, and one containing the conversations
participants = {}
conversations = {}

# Getting info about the participants
The participant_info function extracts information about the participants and stores it in the "participants" dictionary. The keys of the dictionary are the participants' ID numbers. The values are sub-dictionaries that include the participant's role, age, sex, and occupation (if listed). The sub-dictionaries also include the participants' L1s, which are stored in lists (since some participants have multiple L1s).

In [4]:
def participant_info(contents):
    
    #Get all of the participants in the given conversation
    people = contents.find('listPerson', {'type': 'identified'}).findAll('person')
    
    for p in people:
        #info is a subdirector that contains a single participant's information. It will be 
        info = {}
        info['role'] = p['role']
        info['age'] = p.age.get_text()
        info['sex'] = p.sex.get_text()
        
        #In some cases, the occupation isn't listed. If it is included, get the text of the occupation field.
        #If it isn't included, "None" will be stored as the occupation, since p.occupation would return "None."
        try:
            info['occupation'] = p.occupation.get_text()
        except AttributeError:
            info['occupation'] = p.occupation
        
        #Get a list of the languages that the participant speaks. Iterate through the list, and add them to the
        #dictionary according to the speaker's level (ie. L1).
        languages = p.findAll('langKnown')
        for l in languages:
            level = l['level']
            language = l['tag']
        
            if level in info:
                info[level].append(language)
            else:
                info[level] = [language]
    
        #Get the participant's ID number, and make it a key in the participants dictionary. The value will be
        #the info dictionary
        name = p['xml:id']
        participants[name] = info

# Getting lines of the conversation from the file
The conversation_lines function gets each line from the current conversation. The lines are stored as lists in the "conversations" dictionary, whose keys are the names of the XML files. For now, I decided to keep the lines in their XML format; there are a lot of annotations in the XML format that could be useful later on, such as the speaker, pauses, and intonation markings. Converting the XML lines into text/getting rid of the tags is simple, so I could change this later on.

In [5]:
def conversation_lines(file, contents):
    file_name = file.split("/")[-1]
    text_body = contents.body
    xml_lines = text_body.findAll('u')
    conversations[file_name] = xml_lines

# Processing the XML files
This section iterates through all of the files (except for corpus-header.xml) in the VOICE1.0XML/XML directory. It calls conversation_lines and participant_info to extract some important parts of the data from the corpus.

In [6]:
for t in transcripts:
    file = open(t, 'r')
    text = file.read()
    xml_contents = BeautifulSoup(text, 'xml')
    conversation_lines(t, xml_contents)
    participant_info(xml_contents)

In [7]:
conversations['EDcon496.xml'][0]

<u who="#EDcon496_S1" xml:id="EDcon496_u_1"> e<c type="lengthening"/>r leads so <pause/> ma<c type="lengthening"/>n i'm still stuck on lead du<c type="lengthening"/>de <pause dur="PT3S"/></u>

In [8]:
participants['EDcon250_S2']

{'L1': ['ger-AT', 'eng-US'],
 'age': '25-34',
 'occupation': None,
 'role': 'participant',
 'sex': 'female'}

In [9]:
#Get native English speakers
native_speakers = []

#There are multiple ways that English is listed as an L1 ("eng", "eng-US", "eng-CA", "eng-GB", "eng-GY", "eng-AU", etc)
#I used a regular expression to find all of these instances
r = re.compile("eng.*")

for person in participants:
    
    #returns a list of all languages that contain "eng.*" The length of this list should be 1 or 0. If it's 1, the
    #participant has English listed as an L1.
    english = list(filter(r.match, participants[person]['L1']))
    
    if len(english) != 0:
        #print(person, ':', participants[person])
        native_speakers.append(person)

In [10]:
participant = native_speakers[0]
languages = participants[participant]['L1']

In [11]:
english_dialects = {}
for p in native_speakers:
    languages = participants[p]['L1']
    for l in languages:
        if 'eng' not in l:
            continue
        #print(l)
        
        if l not in english_dialects:
            english_dialects[l] = 1
        else:
            english_dialects[l] += 1

In [12]:
english_dialects

{'eng': 8,
 'eng-AU': 3,
 'eng-CA': 6,
 'eng-GB': 39,
 'eng-GY': 1,
 'eng-IE': 3,
 'eng-MT': 7,
 'eng-US': 20}

In [13]:
bilinguals = []
L1_counts = {}
#participants[native_speakers[0]]
for p in participants:
    #print(participants[p]['L1'])
    languages = participants[p]['L1']
    if len(languages) > 1:
        bilinguals.append(p)
        if p in native_speakers:
            if 'eng' not in L1_counts:
                L1_counts['eng'] = 1
            else:
                L1_counts['eng'] += 1
        continue
    
    
    for l in languages:
        L1 = l.split("-")[0]
        if L1 not in L1_counts:
            L1_counts[L1] = 1
        else:
            L1_counts[L1] += 1
        #print(l.split("-")[0])

In [14]:
len(bilinguals)

44

In [15]:
for p in bilinguals:
    print(participants[p]['L1'])

['ger-AT', 'eng-US']
['eng-GY', 'dut-NL']
['fre-FR', 'eng']
['mlt-MT', 'eng-MT']
['mlt-MT', 'eng-MT']
['mlt-MT', 'eng-MT']
['mlt-MT', 'ger-AT']
['ger-AT', 'eng-US']
['ukr-UA', 'rus']
['ger-DE', 'ind-ID']
['ger-AT', 'eng-US']
['eng', 'heb', 'dut']
['slo-SK', 'ger']
['eng-CA', 'chi-CN']
['eng-CA', 'chi-CN']
['fre-CH', 'ger-CH']
['dut-NL', 'eng']
['eng-GB', 'spa-ES']
['fre-CH', 'ger-CH']
['eng-GB', 'spa-ES']
['eng-GB', 'spa-ES']
['eng-GB', 'spa-ES']
['fre-CH', 'ger-CH']
['eng-GB', 'spa-ES']
['eng-GB', 'spa-ES']
['fre-CH', 'ger-CH']
['eng-GB', 'spa-ES']
['fre-CH', 'ger-CH']
['eng-GB', 'spa-ES']
['mlt-MT', 'eng-MT']
['cat-ES', 'spa-ES']
['mlt-MT', 'eng-MT']
['mlt-MT', 'eng-MT']
['mlt-MT', 'eng-MT']
['spa-ES', 'cat-ES']
['dut', 'ger-AT']
['spa-ES', 'cat']
['por-PT', 'ger-AT']
['por-PT', 'ger-AT']
['dut', 'ger-AT']
['ger', 'ita']
['ger-AT', 'pol-PL']
['ara-PS', 'ger-AT']
['per-IR', 'eng-US']


In [16]:
L1_counts

{'alb': 11,
 'ara': 4,
 'arm': 6,
 'bos': 5,
 'bul': 13,
 'cat': 6,
 'chi': 7,
 'cze': 18,
 'dan': 35,
 'dut': 72,
 'eng': 87,
 'est': 8,
 'fin': 51,
 'fre': 63,
 'ger': 303,
 'gre': 14,
 'hin': 4,
 'hun': 13,
 'ice': 3,
 'ind': 2,
 'ita': 54,
 'jpn': 5,
 'kaz': 2,
 'kir': 2,
 'kor': 14,
 'lav': 19,
 'lit': 9,
 'mac': 14,
 'mlt': 22,
 'nor': 34,
 'per': 2,
 'pol': 35,
 'por': 21,
 'rum': 30,
 'rus': 22,
 'scc': 24,
 'scr': 13,
 'slo': 29,
 'slv': 16,
 'spa': 72,
 'swe': 16,
 'tgl': 1,
 'tur': 15,
 'ukr': 4,
 'und': 37,
 'urd': 2,
 'vie': 1,
 'yor': 1}

In [17]:
#conversations['EDcon250.xml']
c = conversations['EDcon4.xml']
unmodified_lines = []
modified_lines = []

for l in c:
    unmodified_lines.append(l.get_text())
    #print(l['who'])       #participants
    #print(l['xml:id'])    #xml_id (conversation line number)
    #print(l.get_text())    #text
    
    #if(l.has_attr("foreign")):
        #print(l)
        
    #line = l.foreign
    #print(line)
  

    if(l['who'] in native_speakers):
        continue
    
    if(l.foreign != None):     #returns the line if has "foreign" tag; None if it doesn't contain the tag
        #print(l.foreign.get_text())   #text of line  
        #print(l.foreign['xml:lang'])  #language used
        #print(l)                      #line (in XML format)
        continue
        
    if(l.unclear != None):     #returns the line if has "unclear" tag; None if it doesn't contain the tag
    #    print(l)
        continue
    
    if(l.reading_aloud != None):
        continue
        #print(l)
        
    if(l.reading != None):
        continue
        #print(l)
        
    modified_lines.append(l)
    
print(len(unmodified_lines))
print(len(modified_lines))


1051
736


In [18]:
modified_conversations = {}
lines = {}

for file in conversations:
    conv_lines = {}
    
    c = conversations[file]
    
    for l in c:
        participant = l['who'].replace("#", "")
        line_id = l['xml:id']
        text = l.get_text()
        
        if participant in native_speakers:
            continue
            
        if participant not in participants:
            #print(participant)
            continue
    
        elif l.foreign != None:     #returns the line if has "foreign" tag; None if it doesn't contain the tag
            #print(l.foreign.get_text())   #text of line  
            #print(l.foreign['xml:lang'])  #language used
            #print(l)                      #line (in XML format)
            continue
        
        elif l.unclear != None:     #returns the line if has "unclear" tag; None if it doesn't contain the tag
            #    print(l)
            continue
    
        elif l.reading_aloud != None:
            continue
            #print(l)
        
        elif l.reading != None :
            continue
            #print(l)
        
        else:
            key = (line_id, participant)
            conv_lines[key] = text
    
    modified_conversations[file] = conv_lines
        
        

In [19]:
#modified_conversations['EDcon4.xml']
#modified_conversations['EDsed364.xml']
modified_conversations['EDsed362.xml']

{('EDsed362_u_10',
  'EDsed362_S2'): ' so  that is going to  teach me  erm  a lesson ',
 ('EDsed362_u_100',
  'EDsed362_S9'): " that their employee can't feed all that people you know  of course er  poverty and of course ",
 ('EDsed362_u_101', 'EDsed362_S10'): ' yes ',
 ('EDsed362_u_102', 'EDsed362_S9'): ' again er ',
 ('EDsed362_u_103',
  'EDsed362_S10'): ' yes  insufficience  of resources so the limitation  some limitations may cause this exploding ',
 ('EDsed362_u_104', 'EDsed362_S12'): ' thank you ',
 ('EDsed362_u_11',
  'EDsed362_S3'): ' okay  er we choose the united nations er because  more or less  all of us we are not very experienced in this topic',
 ('EDsed362_u_110', 'EDsed362_S13'): ' so er we will  also advise the ',
 ('EDsed362_u_112', 'EDsed362_S4'): ' try to guess who ',
 ('EDsed362_u_114',
  'EDsed362_S13'): ' yeah we will also advise the  u n and first  of all erm ',
 ('EDsed362_u_117',
  'EDsed362_S13'): ' the mo-  motivations for terrorism  are political ecological 